# Docker

Comandos principais:

```bash
docker run
docker ps
docker images
```

Exemplo

```bash
docker run -it python /bin/bash
```

Sintaxe:


```
docker <comando docker> <parametros> <imagem docker> <comando a ser executado>
```

- Um container docker é sempre gerado a partir de uma imagem.
- Há diversas versões de imagens
- Exemplo: 
    - [Docker hub: Python](https://hub.docker.com/_/python)
- As imagens podem ter diversas versões
    - [The Startup. Alpine, Slim, Stretch, Buster, Jessie, Bullseye — What are the Differences in Docker Images?](https://medium.com/swlh/alpine-slim-stretch-buster-jessie-bullseye-bookworm-what-are-the-differences-in-docker-62171ed4531d)

```bash
docker pull python:3.11.2-alpine
docker pull python:3.11.2-slim
docker pull python:3.11.2-bullseye
```

```bash
docker images
```

Qual a diferença?

Qual o resultado destes comandos?

```bash
docker run -it python:3.11.2-bullseye /bin/bash -c 'echo "Hello world"'
docker run -it python:3.11.2-slim /bin/bash -c 'echo "Hello world"'
docker run -it python:3.11.2-alpine /bin/bash -c 'echo "Hello world"'
```

```bash
#Cria um container com bash
docker run -it python:3.11.2-slim /bin/bash

> #Streamlit
> pip install streamlit
> git clone https://github.com/darlonv/dockertest.git
```


Problemas?

```bash
#Cria um container com bash
docker run -it python:3.11.2-bullseye /bin/bash

> #Streamlit
> pip install streamlit
> git clone https://github.com/darlonv/dockertest.git
> streamlit run dockertest/app_st.py
```

Agora sim!

Porém...

O container está isolado!  

- Precisamos informar que a porta 8501 pode ser acessada externamente.
- `-p 8501:8501`


```bash
#Cria um container com bash
docker run -it  python:3.11.2-bullseye /bin/bash

> #Streamlit
> pip install streamlit
> git clone https://github.com/darlonv/dockertest.git
> streamlit run --server.address 0.0.0.0 dockertest/app_st.py
```

```bash
docker run -it -p 8501:8501 python:3.11.2-bullseye /bin/bash
```

Em uma única linha:
```bash
docker run -it -p 8501:8501 python:3.11.2-bullseye /bin/bash -c 'pip install streamlit &&  git clone htps://github.com/darlonv/dockertest.git && streamlit run --server.address 0.0.0.0 dockertest/app_st.py'
```

Enquanto o container é executado, em um outro terminal:

```bash
docker ps
```

Identifique o container

```bash
docker pause <nome ou id>
```
    
```bash
docker unpause <nome ou id>
```

```bash
docker ps -a
```
    
- pausar
    `docker stop`
- finalizar
   `docker stop`
- remover/ finalizar e remover
    `docker rm`

```bash
docker ps -a
```

```bash
docker rm <nome ou id>
```

```bash
docker container prune
```
    


outros parametros `docker run`:

```
--rm #remove o container apos finalizar
-v <caminho>:<caminho container> #Monta um diretorio
--name <nome> #Nomeia o container
-ti #Terminal interativo
```

```bash
docker run --rm --name streamlit -t -p 8501:8501 python:3.11.2-bullseye /bin/bash -c 'pip install streamlit &&  git clone htps://github.com/darlonv/dockertest.git && streamlit run --server.address 0.0.0.0 dockertest/app_st.py
```
Observe que agora nao precisamos mais do `i`, já que não há interatividade.

## Dockerfile

- Arquivo `Dockerfile`
- Contém comandos para gerar uma imagem

```bash
FROM python:3.11.2-bullseye                             #Imagem base
RUN pip install streamlit                               #Comando
RUN git clone https://github.com/darlonv/dockertest.git #Comando
WORKDIR /dockertest                                     #Diretório padrão
CMD ["streamlit", "run", "--server.address", "0.0.0.0", "app_st.py"] #Comando a ser executado na criação do container
```

```bash
docker build -t teste .
```

```bash
docker run -p 8501:8501 teste
```

```bash
docker images
```

```bash
docker build -t darlon/dockertest:1.0 .
```

```bash
docker run -it -p 8501:8501 darlon/dockertest
```
erro!


```bash
docker run -it -p 8501:8501 darlon/dockertest:1.0
```

```bash
docker tag darlon/dockertest:1.0 darlon/dockertest:latest
```

```bash
docker images
```

Observe os ids das imagens.

Executamos 
```bash
docker run -it -p 8501:8501 darlon/dockertest
```

Subindo a imagem para o dockerhub

```bash
docker push darlon/dockertest:1.0
```

```bash
docker push darlon/dockertest:latest
```

Em outra máquina, execute:

```bash
docker run -p 8501:8501  darlon/dockertest
```

Remova todas as imagens docker na máquina

```bash
docker images
docker container prune -f
docker rmi <ids>
docker system prune -f
```
    
Execute:
```bash
docker run -p 8501:8501  darlon/dockertest
```
    
Verifique as imagens
```bash
docker images
```

## Atualizando o repositorio

Atualize o projeto no github para a versão 2.0

```python
import streamlit as st

st.title("Dockertest v 2.0")

val = st.text_input('Valor')
val = int(val)
quad = val**2
st.write(f'Quadrado: {quad}')
```

https://docs.streamlit.io/library/api-reference/widgets

```bash
docker build -t darlon/dockertest:2.0 .
docker tag darlon/dockertest:2.0 darlon/dockertest:latest
docker run -p 8501:8501  darlon/dockertest
```

Qual o problema?
- há um erro no código

```python
import streamlit as st

st.title("Dockertest v 2.0")

val = st.text_input('Valor')
if val:
    val = int(val)
    quad = val**2
    st.write(f'Quadrado: {quad}')
```

```bash
docker build -t darlon/dockertest:2.0 .
docker tag darlon/dockertest:2.0 darlon/dockertest:latest
docker push darlon/dockertest:latest
```

```bash
docker run -p 8501:8501  darlon/dockertest
```

Qual o problema?
- O repositório não foi atualizado
    - utilizou a cache

```bash
docker build --no-cache -t darlon/dockertest:2.0 .
docker tag darlon/dockertest:2.0 darlon/dockertest:latest
docker push darlon/dockertest:latest
docker push darlon/dockertest:2.0
docker push darlon/dockertest:latest
docker system prune -f
```

```bash
docker run -p 8501:8501  darlon/dockertest
```